# Multilayer Perceptron

In [117]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [118]:
# Greenplum 4.3.10.0
%sql postgresql://gpdbchina@10.194.10.68:61000/madlib
        
# PostgreSQL local
#%sql postgresql://fmcquillan@localhost:5432/madlib

# Greenplum 4.2.3.0
#%sql postgresql://gpdbchina@10.194.10.68:55000/madlib

u'Connected: gpdbchina@madlib'

In [119]:
%sql select madlib.version();
#%sql select version();

1 rows affected.


version
"MADlib version: 2.0-dev, git revision: rel/v1.12-9-gf790a61, cmake configuration time: Tue Sep 19 17:56:02 UTC 2017, build type: Release, build system: Linux-2.6.18-238.27.1.el5.hotfix.bz516490, C compiler: gcc 4.4.0, C++ compiler: g++ 4.4.0"


# 1.  Create input table for classification

In [120]:
%%sql 
DROP TABLE IF EXISTS iris_data;

CREATE TABLE iris_data(
    id serial,
    attributes numeric[],
    class_text varchar,
    class integer,
    state varchar
);

INSERT INTO iris_data(id, attributes, class_text, class, state) VALUES
(1,ARRAY[5.0,3.2,1.2,0.2],'Iris_setosa',1,'Alaska'),
(2,ARRAY[5.5,3.5,1.3,0.2],'Iris_setosa',1,'Alaska'),
(3,ARRAY[4.9,3.1,1.5,0.1],'Iris_setosa',1,'Alaska'),
(4,ARRAY[4.4,3.0,1.3,0.2],'Iris_setosa',1,'Alaska'),
(5,ARRAY[5.1,3.4,1.5,0.2],'Iris_setosa',1,'Alaska'),
(6,ARRAY[5.0,3.5,1.3,0.3],'Iris_setosa',1,'Alaska'),
(7,ARRAY[4.5,2.3,1.3,0.3],'Iris_setosa',1,'Alaska'),
(8,ARRAY[4.4,3.2,1.3,0.2],'Iris_setosa',1,'Alaska'),
(9,ARRAY[5.0,3.5,1.6,0.6],'Iris_setosa',1,'Alaska'),
(10,ARRAY[5.1,3.8,1.9,0.4],'Iris_setosa',1,'Alaska'),
(11,ARRAY[4.8,3.0,1.4,0.3],'Iris_setosa',1,'Alaska'),
(12,ARRAY[5.1,3.8,1.6,0.2],'Iris_setosa',1,'Alaska'),
(13,ARRAY[5.7,2.8,4.5,1.3],'Iris_versicolor',2,'Alaska'),
(14,ARRAY[6.3,3.3,4.7,1.6],'Iris_versicolor',2,'Alaska'),
(15,ARRAY[4.9,2.4,3.3,1.0],'Iris_versicolor',2,'Alaska'),
(16,ARRAY[6.6,2.9,4.6,1.3],'Iris_versicolor',2,'Alaska'),
(17,ARRAY[5.2,2.7,3.9,1.4],'Iris_versicolor',2,'Alaska'),
(18,ARRAY[5.0,2.0,3.5,1.0],'Iris_versicolor',2,'Alaska'),
(19,ARRAY[5.9,3.0,4.2,1.5],'Iris_versicolor',2,'Alaska'),
(20,ARRAY[6.0,2.2,4.0,1.0],'Iris_versicolor',2,'Alaska'),
(21,ARRAY[6.1,2.9,4.7,1.4],'Iris_versicolor',2,'Alaska'),
(22,ARRAY[5.6,2.9,3.6,1.3],'Iris_versicolor',2,'Alaska'),
(23,ARRAY[6.7,3.1,4.4,1.4],'Iris_versicolor',2,'Alaska'),
(24,ARRAY[5.6,3.0,4.5,1.5],'Iris_versicolor',2,'Alaska'),
(25,ARRAY[5.8,2.7,4.1,1.0],'Iris_versicolor',2,'Alaska'),
(26,ARRAY[6.2,2.2,4.5,1.5],'Iris_versicolor',2,'Alaska'),
(27,ARRAY[5.6,2.5,3.9,1.1],'Iris_versicolor',2,'Alaska'),
(28,ARRAY[5.0,3.4,1.5,0.2],'Iris_setosa',1,'Tennessee'),
(29,ARRAY[4.4,2.9,1.4,0.2],'Iris_setosa',1,'Tennessee'),
(30,ARRAY[4.9,3.1,1.5,0.1],'Iris_setosa',1,'Tennessee'),
(31,ARRAY[5.4,3.7,1.5,0.2],'Iris_setosa',1,'Tennessee'),
(32,ARRAY[4.8,3.4,1.6,0.2],'Iris_setosa',1,'Tennessee'),
(33,ARRAY[4.8,3.0,1.4,0.1],'Iris_setosa',1,'Tennessee'),
(34,ARRAY[4.3,3.0,1.1,0.1],'Iris_setosa',1,'Tennessee'),
(35,ARRAY[5.8,4.0,1.2,0.2],'Iris_setosa',1,'Tennessee'),
(36,ARRAY[5.7,4.4,1.5,0.4],'Iris_setosa',1,'Tennessee'),
(37,ARRAY[5.4,3.9,1.3,0.4],'Iris_setosa',1,'Tennessee'),
(38,ARRAY[6.0,2.9,4.5,1.5],'Iris_versicolor',2,'Tennessee'),
(39,ARRAY[5.7,2.6,3.5,1.0],'Iris_versicolor',2,'Tennessee'),
(40,ARRAY[5.5,2.4,3.8,1.1],'Iris_versicolor',2,'Tennessee'),
(41,ARRAY[5.5,2.4,3.7,1.0],'Iris_versicolor',2,'Tennessee'),
(42,ARRAY[5.8,2.7,3.9,1.2],'Iris_versicolor',2,'Tennessee'),
(43,ARRAY[6.0,2.7,5.1,1.6],'Iris_versicolor',2,'Tennessee'),
(44,ARRAY[5.4,3.0,4.5,1.5],'Iris_versicolor',2,'Tennessee'),
(45,ARRAY[6.0,3.4,4.5,1.6],'Iris_versicolor',2,'Tennessee'),
(46,ARRAY[6.7,3.1,4.7,1.5],'Iris_versicolor',2,'Tennessee'),
(47,ARRAY[6.3,2.3,4.4,1.3],'Iris_versicolor',2,'Tennessee'),
(48,ARRAY[5.6,3.0,4.1,1.3],'Iris_versicolor',2,'Tennessee'),
(49,ARRAY[5.5,2.5,4.0,1.3],'Iris_versicolor',2,'Tennessee'),
(50,ARRAY[5.5,2.6,4.4,1.2],'Iris_versicolor',2,'Tennessee'),
(51,ARRAY[6.1,3.0,4.6,1.4],'Iris_versicolor',2,'Tennessee'),
(52,ARRAY[5.8,2.6,4.0,1.2],'Iris_versicolor',2,'Tennessee');

SELECT * FROM iris_data ORDER BY id;

Done.
Done.
52 rows affected.
52 rows affected.


id,attributes,class_text,class,state
1,"[Decimal('5.0'), Decimal('3.2'), Decimal('1.2'), Decimal('0.2')]",Iris_setosa,1,Alaska
2,"[Decimal('5.5'), Decimal('3.5'), Decimal('1.3'), Decimal('0.2')]",Iris_setosa,1,Alaska
3,"[Decimal('4.9'), Decimal('3.1'), Decimal('1.5'), Decimal('0.1')]",Iris_setosa,1,Alaska
4,"[Decimal('4.4'), Decimal('3.0'), Decimal('1.3'), Decimal('0.2')]",Iris_setosa,1,Alaska
5,"[Decimal('5.1'), Decimal('3.4'), Decimal('1.5'), Decimal('0.2')]",Iris_setosa,1,Alaska
6,"[Decimal('5.0'), Decimal('3.5'), Decimal('1.3'), Decimal('0.3')]",Iris_setosa,1,Alaska
7,"[Decimal('4.5'), Decimal('2.3'), Decimal('1.3'), Decimal('0.3')]",Iris_setosa,1,Alaska
8,"[Decimal('4.4'), Decimal('3.2'), Decimal('1.3'), Decimal('0.2')]",Iris_setosa,1,Alaska
9,"[Decimal('5.0'), Decimal('3.5'), Decimal('1.6'), Decimal('0.6')]",Iris_setosa,1,Alaska
10,"[Decimal('5.1'), Decimal('3.8'), Decimal('1.9'), Decimal('0.4')]",Iris_setosa,1,Alaska


#  2. Classification model

Generate a multilayer perceptron with a single hidden layer of 5 units. Use the attributes column as the independent variables, and use the class column as the classification. Set the tolerance to 0 so that 500 iterations will be run. Use a hyperbolic tangent activation function. The model will be written to mlp_model.

In [121]:
%%sql
DROP TABLE IF EXISTS mlp_model, mlp_model_summary, mlp_model_standardization;

-- Set seed so results are reproducible
SELECT setseed(0);

SELECT madlib.mlp_classification(
    'iris_data',      -- Source table
    'mlp_model',      -- Destination table
    'attributes',     -- Input features
    'class_text',     -- Label
    ARRAY[5],         -- Number of units per layer
    'learning_rate_init=0.003,
    n_iterations=500,
    tolerance=0',     -- Optimizer params
    'tanh',           -- Activation function
    NULL,             -- Default weight (1)
    FALSE,            -- No warm start
    FALSE             -- Not verbose
);

Done.
1 rows affected.
1 rows affected.


mlp_classification
""


# 3.  View the classification model

In [122]:
%%sql
SELECT * FROM mlp_model;

1 rows affected.


coeff,loss,num_iterations
"[0.418378455147, 0.0769385374681, -0.146527483484, 0.998888382223, 1.12987089952, -0.437290718473, -0.409273157364, 0.498986904082, -0.544260518783, -0.0914464021734, 0.0390525613337, 0.288450666036, -0.327040616284, 0.16574382417, 0.872753267348, 0.675971654762, 0.318207842514, 0.290765083843, -0.468849343505, 0.288355834648, -0.773220988598, -0.277246576994, -0.676925299898, 0.669871211106, -0.391874491507, 0.0738458598585, -1.85544863789, -0.0153495613681, -0.111982901002, -0.225221594881, -0.217849875552, 0.392310858189, 1.37852218837, -1.08667184295, 1.29050890203, -0.251494108369, 0.229914016792]",0.0105868627707,500


# 4.  Classification with n_tries
Now, use the n_tries optimizer parameter to learn and choose the best model among n_tries number of models learned by the algorithm. Run only for 100 iterations and choose the best model from this short run.

In [123]:
%%sql
DROP TABLE IF EXISTS mlp_model, mlp_model_summary, mlp_model_standardization;

-- Set seed so results are reproducible
SELECT setseed(0);

SELECT madlib.mlp_classification(
    'iris_data',      -- Source table
    'mlp_model',      -- Destination table
    'attributes',     -- Input features
    'class_text',     -- Label
    ARRAY[5],         -- Number of units per layer
    'learning_rate_init=0.003,
    n_iterations=50,
    tolerance=0,
    n_tries=3',       -- Optimizer params, with n_tries
    'tanh',           -- Activation function
    NULL,             -- Default weight (1)
    FALSE,            -- No warm start
    FALSE             -- Not verbose
);

Done.
1 rows affected.
1 rows affected.


mlp_classification
""


# 5.  View the classification model with n_tries

In [124]:
%%sql
SELECT * FROM mlp_model;

1 rows affected.


coeff,loss,num_iterations
"[0.386129091572, 0.0230804544453, 0.0360373350657, 0.85734404404, 1.00624019669, -0.511165317414, -0.256735693588, 0.424597795202, -0.344940825214, 0.105701317173, -0.0239838319549, 0.19433093383, -0.198067795405, -0.00423615288994, 0.714494593542, 0.67620771672, 0.31692178694, 0.291634997959, -0.470338692309, 0.286903795447, -0.797640757541, -0.36833411352, -0.623711536473, 0.556232364597, -0.501937062704, 0.0664559736029, -1.29169165842, -0.39640660159, 0.349410636066, -0.266301858512, -0.12933320224, 0.399700744444, 0.814765208894, -0.705614802744, 0.829115365015, -0.210413844738, 0.14139734348]",0.169187585083,50


# 6. Classification with warm start
Next, use the warm_start parameter to start learning a new model, using the coefficients already present in mlp_model. Note that we must not drop the mlp_model table, and cannot use the n_tries parameter if warm_start is used.

In [126]:
%%sql
SELECT madlib.mlp_classification(
    'iris_data',      -- Source table
    'mlp_model',      -- Destination table
    'attributes',     -- Input features
    'class_text',     -- Label
    ARRAY[5],         -- Number of units per layer
    'learning_rate_init=0.003,
    n_iterations=450,
    tolerance=0',     -- Optimizer params
    'tanh',           -- Activation function
    NULL,             -- Default weight (1)
    TRUE,             -- Warm start
    FALSE             -- Not verbose
);

1 rows affected.


mlp_classification
""


# 7.  View the classification model with warm_start
Notice that the loss is lower compared to the previous example run of 500 iterations without n_tries, despite having the same values for every other parameter. This is because the algorithm learnt three different models starting with a different set of initial weights for the coefficients, and chose the best model among them.

In [127]:
%%sql
SELECT * FROM mlp_model;

1 rows affected.


coeff,loss,num_iterations
"[0.43089489035, 0.0840880267225, -0.180463790458, 1.02521696861, 1.15327056306, -0.425505368961, -0.445320886704, 0.51333393744, -0.594555309569, -0.140856773835, 0.0616746727506, 0.306834150297, -0.359872964248, 0.206830852263, 0.911186393447, 0.675568040494, 0.316470613819, 0.291871023206, -0.471193550262, 0.286075681893, -0.759796508813, -0.252763378948, -0.690187333341, 0.70213228757, -0.360741864941, 0.0646874010451, -1.97963455591, 0.0877591861911, -0.220936503136, -0.222359882677, -0.244113625762, 0.401469317001, 1.50270810639, -1.18978059055, 1.39946250414, -0.254355820573, 0.256177767002]",0.00513279402477,450


# 8. Classification with grouping
Next, group the training data by state, and learn a different model for each state.

In [128]:
%%sql
DROP TABLE IF EXISTS mlp_model_group, mlp_model_group_summary, mlp_model_group_standardization;

-- Set seed so results are reproducible
SELECT setseed(0);

SELECT madlib.mlp_classification(
    'iris_data',        -- Source table
    'mlp_model_group',  -- Destination table
    'attributes',       -- Input features
    'class_text',       -- Label
    ARRAY[5],           -- Number of units per layer
    'learning_rate_init=0.003,
    n_iterations=50,
    tolerance=0,
    n_tries=3',         -- Optimizer params, with n_tries
    'tanh',             -- Activation function
    NULL,               -- Default weight (1)
    FALSE,              -- No warm start
    FALSE,              -- Not verbose
    'state'             -- Grouping column
);

Done.
1 rows affected.
1 rows affected.


mlp_classification
""


# 9.  View the classification model with grouping
A separate model is learnt for each state, and the result table displays the name of the state (grouping column) associated with the model.

In [129]:
%%sql
SELECT * FROM mlp_model_group ORDER BY state;

2 rows affected.


state,coeff,loss,num_iterations
Alaska,"[0.361442205117, 0.0437833983169, 0.0228267239333, 0.86051976995, 1.00898866341, -0.509493068163, -0.259717661563, 0.423128001757, -0.348520276299, 0.102569227429, -0.023820253367, 0.194242907683, -0.194973841963, -0.00354279848888, 0.715171987578, 0.679803198566, 0.30085527398, 0.290398225819, -0.481339584921, 0.273581567191, -0.798021118384, -0.364181329645, -0.631718355895, 0.568550659826, -0.492243523831, 0.0678832140837, -1.29021370112, -0.396961890891, 0.347728139082, -0.247139454296, -0.165073883437, 0.398273503965, 0.813287251616, -0.705059513443, 0.830797861999, -0.229576248952, 0.177138024677]",0.169694760351,50
Tennessee,"[0.286460719233, -0.391554641774, 0.647611140251, -0.670341290996, 0.111261651571, -0.323445227135, -0.362568050061, 0.556843820772, 0.542593756362, -0.849092718814, 0.639022610897, 0.289608013322, -0.771514626336, -0.225524049129, 0.55654170876, 0.699286372946, -0.0971044516431, -0.856859889972, -0.53385894952, 0.181718585915, -0.635491855087, -0.334708753092, 0.676444860354, 0.798824989025, 0.712555678327, -0.362929653511, 0.347434351706, 0.177418782808, 0.225111014926, 0.178241297519, -0.0229063368163, 0.0404201612098, -1.12689954392, -0.523963295766, 1.0563791555, 0.642674420838, 0.742594634114]",0.159575522026,50


# 10.  Classification with warm start and grouping
Next use warm_start to improve the models that are already present in the mlp_model_group table. Note again that we cannot use n_tries with warm_start, and the model table must already be present.

In [130]:
%%sql
SELECT madlib.mlp_classification(
    'iris_data',        -- Source table
    'mlp_model_group',  -- Destination table
    'attributes',       -- Input features
    'class_text',       -- Label
    ARRAY[5],           -- Number of units per layer
    'learning_rate_init=0.003,
    n_iterations=450,
    tolerance=0',       -- Optimizer params
    'tanh',             -- Activation function
    NULL,               -- Default weight (1)
    TRUE,               -- Warm start
    FALSE,              -- Not verbose
    'state'             -- Grouping column
);

1 rows affected.


mlp_classification
""


# 11. View the classification model with warm start and grouping


In [132]:
%%sql
SELECT * FROM mlp_model_group ORDER BY state;

2 rows affected.


state,coeff,loss,num_iterations
Alaska,"[0.372342289818, 0.0954960560128, -0.153810124763, 1.00725097032, 1.13495556432, -0.429533598061, -0.409198527269, 0.478752722727, -0.551271316257, -0.0950828556324, 0.0237081636546, 0.297325734546, -0.307808869913, 0.17901470615, 0.882685356116, 0.67682803433, 0.28787018652, 0.297557142865, -0.499522323052, 0.256295844717, -0.768404886254, -0.266707916954, -0.676330194506, 0.699875226902, -0.36878024444, 0.0723594672762, -1.86161184866, -0.0295697622013, -0.116698438374, -0.188190989233, -0.253894999257, 0.393797250761, 1.38468539912, -1.07245164218, 1.29522443949, -0.288524714015, 0.265959140497]",0.0106816837098,450
Tennessee,"[0.101408120906, -0.504692476376, 0.754172746712, -0.859366119317, -0.0883419348625, -0.322217245475, -0.488849126018, 0.640836955763, 0.379240951714, -1.01591620805, 0.561633386233, 0.474881549745, -0.820848160685, -0.0367588935663, 0.756738298906, 0.690992107944, -0.0117938780722, -0.911609074051, -0.417367078805, 0.303153350896, -0.502205494032, -0.262138409583, 0.531455906745, 0.931199625293, 0.837077534104, -0.383150148885, 0.722094737958, 0.54110242729, -0.140304542646, 0.107821427424, -0.229319721716, 0.0606406565821, -1.50155993006, -0.887646940248, 1.42179471312, 0.713094290933, 0.949008019012]",0.00853723754598,450


# 12. Create input data for regression

This dataset contains housing prices.

In [133]:
%%sql
DROP TABLE IF EXISTS lin_housing;

CREATE TABLE lin_housing (id serial, 
                          x numeric[], 
                          zipcode int, 
                          y float8);

INSERT INTO lin_housing(id, x, zipcode, y) VALUES
(1,ARRAY[1,0.00632,18.00,2.310,0,0.5380,6.5750,65.20,4.0900,1,296.0,15.30,396.90,4.98],94016,24.00),
(2,ARRAY[1,0.02731,0.00,7.070,0,0.4690,6.4210,78.90,4.9671,2,242.0,17.80,396.90,9.14],94016,21.60),
(3,ARRAY[1,0.02729,0.00,7.070,0,0.4690,7.1850,61.10,4.9671,2,242.0,17.80,392.83,4.03],94016,34.70),
(4,ARRAY[1,0.03237,0.00,2.180,0,0.4580,6.9980,45.80,6.0622,3,222.0,18.70,394.63,2.94],94016,33.40),
(5,ARRAY[1,0.06905,0.00,2.180,0,0.4580,7.1470,54.20,6.0622,3,222.0,18.70,396.90,5.33],94016,36.20),
(6,ARRAY[1,0.02985,0.00,2.180,0,0.4580,6.4300,58.70,6.0622,3,222.0,18.70,394.12,5.21],94016,28.70),
(7,ARRAY[1,0.08829,12.50,7.870,0,0.5240,6.0120,66.60,5.5605,5,311.0,15.20,395.60,12.43],94016,22.90),
(8,ARRAY[1,0.14455,12.50,7.870,0,0.5240,6.1720,96.10,5.9505,5,311.0,15.20,396.90,19.15],94016,27.10),
(9,ARRAY[1,0.21124,12.50,7.870,0,0.5240,5.6310,100.00,6.0821,5,311.0,15.20,386.63,29.93],94016,16.50),
(10,ARRAY[1,0.17004,12.50,7.870,0,0.5240,6.0040,85.90,6.5921,5,311.0,15.20,386.71,17.10],94016,18.90),
(11,ARRAY[1,0.22489,12.50,7.870,0,0.5240,6.3770,94.30,6.3467,5,311.0,15.20,392.52,20.45],94016,15.00),
(12,ARRAY[1,0.11747,12.50,7.870,0,0.5240,6.0090,82.90,6.2267,5,311.0,15.20,396.90,13.27],20001,18.90),
(13,ARRAY[1,0.09378,12.50,7.870,0,0.5240,5.8890,39.00,5.4509,5,311.0,15.20,390.50,15.71],20001,21.70),
(14,ARRAY[1,0.62976,0.00,8.140,0,0.5380,5.9490,61.80,4.7075,4,307.0,21.00,396.90,8.26],20001,20.40),
(15,ARRAY[1,0.63796,0.00,8.140,0,0.5380,6.0960,84.50,4.4619,4,307.0,21.00,380.02,10.26],20001,18.20),
(16,ARRAY[1,0.62739,0.00,8.140,0,0.5380,5.8340,56.50,4.4986,4,307.0,21.00,395.62,8.47],20001,19.90),
(17,ARRAY[1,1.05393,0.00,8.140,0,0.5380,5.9350,29.30,4.4986,4,307.0,21.00,386.85,6.58],20001, 23.10),
(18,ARRAY[1,0.78420,0.00,8.140,0,0.5380,5.9900,81.70,4.2579,4,307.0,21.00,386.75,14.67],20001,17.50),
(19,ARRAY[1,0.80271,0.00,8.140,0,0.5380,5.4560,36.60,3.7965,4,307.0,21.00,288.99,11.69],20001,20.20),
(20,ARRAY[1,0.72580,0.00,8.140,0,0.5380,5.7270,69.50,3.7965,4,307.0,21.00,390.95,11.28],20001,18.20);

SELECT * FROM lin_housing ORDER BY id;

Done.
Done.
20 rows affected.
20 rows affected.


id,x,zipcode,y
1,"[Decimal('1'), Decimal('0.00632'), Decimal('18.00'), Decimal('2.310'), Decimal('0'), Decimal('0.5380'), Decimal('6.5750'), Decimal('65.20'), Decimal('4.0900'), Decimal('1'), Decimal('296.0'), Decimal('15.30'), Decimal('396.90'), Decimal('4.98')]",94016,24.0
2,"[Decimal('1'), Decimal('0.02731'), Decimal('0.00'), Decimal('7.070'), Decimal('0'), Decimal('0.4690'), Decimal('6.4210'), Decimal('78.90'), Decimal('4.9671'), Decimal('2'), Decimal('242.0'), Decimal('17.80'), Decimal('396.90'), Decimal('9.14')]",94016,21.6
3,"[Decimal('1'), Decimal('0.02729'), Decimal('0.00'), Decimal('7.070'), Decimal('0'), Decimal('0.4690'), Decimal('7.1850'), Decimal('61.10'), Decimal('4.9671'), Decimal('2'), Decimal('242.0'), Decimal('17.80'), Decimal('392.83'), Decimal('4.03')]",94016,34.7
4,"[Decimal('1'), Decimal('0.03237'), Decimal('0.00'), Decimal('2.180'), Decimal('0'), Decimal('0.4580'), Decimal('6.9980'), Decimal('45.80'), Decimal('6.0622'), Decimal('3'), Decimal('222.0'), Decimal('18.70'), Decimal('394.63'), Decimal('2.94')]",94016,33.4
5,"[Decimal('1'), Decimal('0.06905'), Decimal('0.00'), Decimal('2.180'), Decimal('0'), Decimal('0.4580'), Decimal('7.1470'), Decimal('54.20'), Decimal('6.0622'), Decimal('3'), Decimal('222.0'), Decimal('18.70'), Decimal('396.90'), Decimal('5.33')]",94016,36.2
6,"[Decimal('1'), Decimal('0.02985'), Decimal('0.00'), Decimal('2.180'), Decimal('0'), Decimal('0.4580'), Decimal('6.4300'), Decimal('58.70'), Decimal('6.0622'), Decimal('3'), Decimal('222.0'), Decimal('18.70'), Decimal('394.12'), Decimal('5.21')]",94016,28.7
7,"[Decimal('1'), Decimal('0.08829'), Decimal('12.50'), Decimal('7.870'), Decimal('0'), Decimal('0.5240'), Decimal('6.0120'), Decimal('66.60'), Decimal('5.5605'), Decimal('5'), Decimal('311.0'), Decimal('15.20'), Decimal('395.60'), Decimal('12.43')]",94016,22.9
8,"[Decimal('1'), Decimal('0.14455'), Decimal('12.50'), Decimal('7.870'), Decimal('0'), Decimal('0.5240'), Decimal('6.1720'), Decimal('96.10'), Decimal('5.9505'), Decimal('5'), Decimal('311.0'), Decimal('15.20'), Decimal('396.90'), Decimal('19.15')]",94016,27.1
9,"[Decimal('1'), Decimal('0.21124'), Decimal('12.50'), Decimal('7.870'), Decimal('0'), Decimal('0.5240'), Decimal('5.6310'), Decimal('100.00'), Decimal('6.0821'), Decimal('5'), Decimal('311.0'), Decimal('15.20'), Decimal('386.63'), Decimal('29.93')]",94016,16.5
10,"[Decimal('1'), Decimal('0.17004'), Decimal('12.50'), Decimal('7.870'), Decimal('0'), Decimal('0.5240'), Decimal('6.0040'), Decimal('85.90'), Decimal('6.5921'), Decimal('5'), Decimal('311.0'), Decimal('15.20'), Decimal('386.71'), Decimal('17.10')]",94016,18.9


# 13. Regression model
Now train a regression model using a multilayer perceptron with 2 hidden layers of 25 nodes each.

In [134]:
%%sql
DROP TABLE IF EXISTS mlp_regress, mlp_regress_summary, mlp_regress_standardization;

SELECT setseed(0);
SELECT madlib.mlp_regression(
    'lin_housing',    -- Source table
    'mlp_regress',    -- Desination table
    'x',              -- Input features
    'y',              -- Dependent variable
    ARRAY[25,25],     -- Number of units per layer
    'learning_rate_init=0.001,
    n_iterations=500,
    lambda=0.001,
    tolerance=0',     -- Optimizer params
    'relu',           -- Activation function
    NULL,             -- Default weight (1)
    FALSE,            -- No warm start
    FALSE             -- Not verbose
);

Done.
1 rows affected.
1 rows affected.


mlp_regression
""


# 14. View the regression model

In [135]:
%%sql
SELECT * FROM mlp_regress;

1 rows affected.


[([0.752109182538, -0.0824025290857, 0.0410059928624, 0.337021944674, 0.65416507111, -0.235970675026, 0.105710990669, 0.0477997938709, -0.25104991458, -0.603068658656, 0.383645101517, 0.401628129242, -0.162802028669, -0.156557103874, 0.2105522042, 0.44411313857, 0.105882403567, 0.267663469827, -0.399241363581, -0.0170604141314, -0.377382699093, -0.299853370639, 0.126338917041, 0.210676270596, -0.070565829362, -0.0757525460553, -0.401460560938, -0.210582542703, 0.383367516914, -0.268075915715, -0.173442916036, 0.264575649181, 0.184456779405, -0.0097165532605, 0.00997431792943, 0.0189488858382, 0.116511649746, 0.558767817585, -0.351001044668, 0.604559944202, 0.224654308969, 0.329122775384, -0.136962649643, 0.35692433053, -0.408651008128, -0.0575033890757, 0.24008692791, 0.375288250556, -0.382268762394, 0.285071651184, 0.0202815938214, -0.365421119429, -0.0645604701344, 0.0781523727731, 0.323576095594, -0.131978158405, -0.393724869374, -0.332825704434, -0.0441813385304, -0.417956144869, -0.256460714895, 0.367189143298, 0.511310537138, 0.214672345403, -0.226875489378, 0.0310210509308, -0.118906658933, 0.344708505926, -0.17563889882, 0.336038890156, -0.0667462014994, -0.405936668753, 0.0138491914875, 0.144754149015, 0.616079043975, 0.130704353611, -0.168294467682, 0.392677221941, -0.00288123069978, -0.0508927001029, 0.146569613764, -0.245656287204, -0.00855988100946, 0.231879433703, 0.214937925868, -0.237298057184, -0.221856092555, 0.419855473628, -0.184932398926, 0.509549986398, 0.848364676338, 0.0691571059329, -0.12501098348, 0.354573894926, -0.0312632006993, 0.330781584014, -0.0945575781122, 0.599881646941, 0.477930952035, 0.0368644507092, 0.291402370717, -0.178220047967, 0.25696983585, 0.423955762121, -0.224823810899, 0.117759914943, 0.110071199153, 0.165406320093, 0.000169648304621, -0.0994666021946, 0.22313907609, -0.212569531367, -0.00804181774423, -0.161692425466, -0.0795345385668, -0.414853143099, -0.0319878410506, -0.0228943999305, -0.403365320244, 0.827452938969, -0.241468133377, -0.291736333618, 0.128741060377, 0.147398430131, 0.420266409293, 0.339390474256, 0.197863253743, -0.0954143221801, 0.162946507755, -0.105789077643, -0.151555115552, -0.171680803545, 0.148136907649, -0.327476021688, -0.218371335448, 0.479974277928, -0.292204793018, 0.0691508765986, -0.136512537546, 0.34812612476, -0.331952205898, 0.494087559139, -0.449934439832, 0.0275333229132, -0.655728766517, 0.00147701174048, 0.41330862736, 0.0922963507921, 0.00128558582822, 0.352445255694, 0.266983248469, 0.202645340868, -0.466875319026, -0.274237283821, -0.0344874375585, -0.32906363626, -0.390803035776, -0.0975552077039, -0.137380758754, 0.0485297845885, 0.0880149390821, 0.15598584193, -0.381638091268, -0.349829147079, 0.187767379305, -0.19627281968, 0.304243720029, -0.440061372901, 0.513066236833, -0.551600207373, 0.289095512602, -0.28223824147, -0.544441635982, 0.402488722797, -0.0610017651608, 0.050843874836, -0.208241178518, -0.131554252864, 0.320678048538, 0.312675207357, -0.0355083214795, 0.0409488582943, 0.0599442092144, -0.102774575851, 0.119165691458, -0.312361157743, -0.0604515759764, 0.328759826876, -0.164492431141, 0.381209091231, -0.0199720553027, 0.207546700513, 0.219121807697, 0.130512901404, -0.421054928057, 0.331651621998, 0.259992989232, 0.503678285778, 0.043267841401, 0.0763523948028, 0.374054908823, 0.0306548439025, 0.837393801255, 0.337209566495, 0.245463036569, -0.0771582704414, -0.467210091513, 0.475084836639, 0.323805117209, -0.383025045521, 0.149104049736, -0.196576189906, 0.0857511302967, -0.226165994054, 0.0930391937861, -0.142880412787, -0.198743603174, -0.00155007277353, -0.406210296892, 0.227628316285, -0.185312440189, 0.0599849059358, 0.00114737047939, 0.358812682523, -0.234918996681, 0.0341316200714, 0.171073526446, -0.265833332355, -0.0338383472716, 0.100681791486, 0.346657350338, -0.00280320436295, -0.143089617661, 0.267059348013, -0.0749074502189, -0.35274511192, -0.0881180093039, 0.103616756818, 0.243472935978, -0.2121845

# 15. Regression with n_tries
Now, use the n_tries optimizer parameter to learn and choose the best model among n_tries number of models learnt by the algorithm.

In [136]:
%%sql
DROP TABLE IF EXISTS mlp_regress, mlp_regress_summary, mlp_regress_standardization;

-- Set seed so results are reproducible
SELECT setseed(0);

SELECT madlib.mlp_regression(
    'lin_housing',    -- Source table
    'mlp_regress',    -- Desination table
    'x',              -- Input features
    'y',              -- Dependent variable
    ARRAY[25,25],     -- Number of units per layer
    'learning_rate_init=0.001,
    n_iterations=50,
    n_tries=3,
    lambda=0.001,
    tolerance=0',     -- Optimizer params, with n_tries
    'relu',           -- Activation function
    NULL,             -- Default weight (1)
    FALSE,            -- No warm start
    FALSE             -- Not verbose
);

Done.
1 rows affected.
1 rows affected.


mlp_regression
""


# 16. View the regression model with n_tries

In [137]:
%%sql
SELECT * FROM mlp_regress;

1 rows affected.


[([-0.267884091975, -0.219645368263, 0.231191666056, 0.112210663859, -0.299612439604, -0.0301941289693, -0.0388095807194, -0.0543356931718, 0.0948035980801, 0.247617904056, 0.0636258331373, -0.198783733323, 0.231350247746, 0.129124726099, -0.315242475597, 0.699935790946, -0.0194724449908, -0.297612323736, 0.019574815136, 0.493608921544, 0.326826584562, -0.258483394154, 0.279480531471, 0.140339073845, -0.260807275504, 0.470266552063, 0.446354947979, -0.472281932244, 0.342001931949, 0.234098953251, 0.140209085594, -0.388312800384, -0.290866550994, -0.127956220414, 0.171644894652, -0.186690276902, 0.263469393078, -0.45291234856, 0.117419779071, -0.130784630811, 0.375554987908, -0.088132691869, 0.022480559778, 0.211755356068, 0.313195493059, 0.335696820067, 0.10804520696, 0.100050802863, -0.576819925543, 0.526311827534, -0.106842711943, 0.436268287984, -0.284782715826, -0.27134111425, 0.166691115833, -0.201950133537, -0.186357413619, 0.412896033548, 0.0285505713215, -0.592295787575, 0.501658431042, 0.30539098292, -0.726732235638, 0.431432977297, -0.217229322519, 0.10913680524, -0.417607243842, 0.362441496248, -0.00286696559387, 0.267372763319, -0.140547099758, -0.0267360705434, -0.506695024119, 0.358575176179, -0.408068693191, -0.263107141043, -0.0370997293541, 0.26886341778, 0.0593132909431, -0.0182365342891, 0.279764640825, 0.341604948366, -0.099658898947, 0.39485691795, -0.260354037381, -0.228564814349, -0.174924817917, -0.125426423375, 0.348631010164, 0.106391143527, -0.239247711595, 0.0905492351195, 0.130833007902, -0.158682690547, -0.00832785981131, 0.15217459842, -0.334226513365, 0.140334646064, 0.0123476455566, 0.0853336980552, -0.0192939818629, 0.322564157367, -0.104381521101, 0.0860517850725, 0.166663246649, -0.0568740967012, 0.225984077925, -0.251964089417, -0.107434200446, -0.0586781116335, 0.108123583157, -0.210131142199, -0.225452908089, -0.36397341041, -0.290790574662, -0.0248891692802, -0.173063611379, -0.116968873441, 0.203924901749, -0.25341011612, -0.351650047003, -0.278375373051, 0.165663172135, -0.000354851059424, -0.00613478914036, -0.198369437959, -0.265380238417, 0.0804367990652, 0.301228048339, 0.27777477726, -0.244223015311, -0.113866366935, 0.189953196583, -0.0109234771153, -0.352920492246, -0.120309474042, 0.345950803795, 0.267710213713, 0.101049602656, -0.223983142942, -0.223653752893, -0.228074823648, -0.016997618676, -0.0255612529949, -0.150096085974, 0.0338172221999, 0.242591643748, -0.0013147808096, 0.339131037962, 0.121262189344, 0.202841007643, 0.380242040767, 0.277838389902, -0.15634643657, 0.0623215631207, -0.139204827221, 0.137640664899, 0.224100770834, 0.264211079996, 0.012718477608, 0.046723596152, -0.297654875086, 0.434072897858, -0.0592987039355, -0.0255362473064, -0.395058441243, 0.225172995843, 0.0449617228275, 0.312622891679, -0.0263450359428, 0.222778045941, -0.268810602189, 0.135480597176, -0.176964788065, 0.0752479672553, -0.362946630956, 0.265588530971, 0.000878720353017, 0.00780926919555, 0.247144831597, -0.189669062058, -0.260895335453, 0.108056718579, 0.454673023774, 0.094404739258, -0.163525629318, -0.166469369324, -0.341437829626, -0.42687062782, -0.304516612636, 0.284502334771, 0.158537657202, -0.350616751148, 0.184502362373, 0.347939106695, 0.445402575718, 0.173695585365, 0.171417133809, -0.182631840522, 0.207905932191, -0.271165937073, -0.250869702795, 0.169503431579, 0.236242344435, -0.207547488588, -0.0364518371243, 0.357851282757, -0.229670891813, 0.342419311194, -0.205164026312, 0.444126705863, 0.37747722625, 0.0489863168843, 0.153057891065, 0.392770395182, -0.302482172087, -0.340458904562, -0.264220974137, -0.206942845269, -0.361163583255, -0.141116184999, -0.310176942567, 0.138959005486, 0.0642488315536, 0.0886781341373, 0.279957753628, -0.182575294176, -0.182464400078, -0.121326279794, 0.0702876520052, -0.328994493469, -0.0910885035831, 0.156216498264, 0.319237242688, -0.129421946875, 0.391582977561, -0.236857379772, -0.310114164963, -0.250796120155, -0.375120112056, 

# 17. Regression with warm start
Next, use the warm_start parameter to start learning a new model, using the coefficients already present in mlp_regress.

In [138]:
%%sql
SELECT madlib.mlp_regression(
    'lin_housing',    -- Source table
    'mlp_regress',    -- Desination table
    'x',              -- Input features
    'y',              -- Dependent variable
    ARRAY[25,25],     -- Number of units per layer
    'learning_rate_init=0.001,
    n_iterations=450,
    lambda=0.001,
    tolerance=0',     -- Optimizer params
    'relu',           -- Activation function
    NULL,             -- Default weight (1)
    TRUE,             -- Warm start
    FALSE             -- Not verbose
);

1 rows affected.


mlp_regression
""


# 18. View regression model with warm start 

In [139]:
%%sql
SELECT * FROM mlp_regress;

1 rows affected.


[([-0.269202990986, -0.218570645484, 0.2301337218, 0.112717475137, -0.294946458088, -0.0300463893758, -0.0356307926074, -0.0491008854558, 0.0815488684928, 0.244784800755, 0.0644759293468, -0.194715077712, 0.230065966166, 0.126581447074, -0.321773718669, 0.540694489072, -0.0193771664979, -0.532671090652, -0.0313011517908, 0.457542831831, 0.325227425107, -0.266083562069, 0.462947899142, 0.129634920668, -0.538878415199, 0.484827595211, 0.42360272994, -0.411602054674, 0.271771578486, 0.0596548925852, 0.0791660838636, -0.386412789442, -0.488872120776, -0.136541534317, 0.13751826488, -0.185776803107, 0.22881712789, -0.548988265064, 0.267068967217, -0.292051921193, 0.339285647911, -0.1238888207, 0.0206317784922, 0.216069000394, 0.329886450794, 0.206506960845, 0.107516542765, 0.0976826710817, -0.470470616752, 0.738321228984, -0.106319931546, 0.570853353829, 0.0457221369977, -0.400533886326, 0.373617272107, -0.170490295647, -0.0223357098149, 0.32585204228, 0.0911899038463, -0.659252601867, 0.422515729382, 0.303896707671, -0.853471142627, 0.369023808111, -0.231166037961, 0.108602799865, -0.414522644797, 0.266731043306, 0.48211305224, -0.207833961739, -0.0792220595003, -0.0328667424358, -0.434503558442, 0.269319009127, -0.218732054641, -0.281087688671, -0.0369182007201, 0.129865993931, 0.0972765630666, 0.000286491875638, 0.278395755027, 0.367300938119, -0.134806659458, 0.515370518178, -0.459687762432, -0.146161522816, -0.133996644341, -0.154048979411, 0.523644457782, -0.00483319924512, -0.500566269458, 0.0901061785507, 0.527638968645, -0.222857429646, -0.0080340559357, 0.151430009536, -0.327631567653, 0.542668350688, -0.0644904374918, 0.341230289214, -0.131169008795, 0.298761026496, -0.0308833294804, -0.189041543666, 0.852833441295, -0.0777878284323, 0.224878339926, -0.238844352625, -0.118525327356, -0.0631456512246, 0.107594535466, -0.214164136225, -0.179102697478, -0.358683773247, -0.282402698885, -0.0223830094083, -0.178638369859, -0.105726290211, 0.191864680396, -0.267644990203, -0.359234958781, -0.277013284934, 0.186031893118, 0.0057313755493, -0.0131902228328, -0.197398818145, -0.25392781438, 0.113024049358, 0.350916588121, 0.282033411821, -0.228420524731, -0.104684811081, 0.190696293192, -0.0138690134168, -0.318256534295, -0.140349091022, 0.344258069727, 0.276565160472, 0.110160797243, -0.245273414317, -0.222559417155, -0.238001200322, -0.131502501225, -0.0345545463754, -0.205455458889, 0.0107961315023, 0.228093461312, -0.0124330071827, 0.288892784207, 0.238311942753, 0.141806561091, 0.378381519964, 0.283441719927, -0.138729103304, 0.0292581495577, -0.138523699299, 0.142080122974, 0.365864645689, 0.264861399476, 0.140242278296, 0.062366348591, -0.29418047471, 0.425865843396, -0.0223300856505, -0.0218205800243, -0.389131455164, 0.224071226459, 0.0390094598018, 0.322555154992, -0.0376843086717, 0.221687995026, -0.262902683254, 0.139642219959, -0.176760243104, 0.0673999132088, -0.37579162631, 0.26556268314, -0.0055129369382, 0.0102609572785, 0.23995478307, -0.282462604997, -0.259618776996, -0.121107114572, 0.462391035884, 0.0608471714245, -0.162725499924, -0.227352885032, -0.479758034562, -0.21402162415, -0.532152332953, 0.272460178997, 0.108130901184, -0.38891539012, 0.138106685466, 0.37309914945, 0.4028624544, 0.172845694478, 0.038271157473, -0.182288447831, 0.194601213581, -0.269839124653, -0.255327170785, 0.270401633068, 0.337599953332, -0.334409951211, -0.00099107645653, 0.353774363472, -0.228137962086, 0.340368962329, -0.189791397898, 0.440626514439, 0.375630233673, 0.0225846152813, 0.176325306514, 0.3866096753, -0.301002129619, -0.341082905678, -0.195269993419, -0.200296723469, -0.282954711354, -0.11693615015, -0.303781467128, 0.112132063618, 0.0873727625725, 0.0486731978826, 0.174034398739, -0.181681954953, -0.343109243326, -0.0572858138991, 0.0158898734601, -0.327384726457, -0.11719371668, 0.111638719548, 0.514973225013, -0.312275391523, 0.443683579911, -0.246956233013, -0.37875140803, -0.282155898591, -0.290388253706, 0.196

# 19. Regression with grouping
Next, group the training data by zipcode, and learn a different model for each zipcode.

In [140]:
%%sql
DROP TABLE IF EXISTS mlp_regress_group, mlp_regress_group_summary, mlp_regress_group_standardization;

-- Set seed so results are reproducible
SELECT setseed(0);

SELECT madlib.mlp_regression(
    'lin_housing',    -- Source table
    'mlp_regress_group',    -- Desination table
    'x',              -- Input features
    'y',              -- Dependent variable
    ARRAY[25,25],     -- Number of units per layer
    'learning_rate_init=0.001,
    n_iterations=500,
    lambda=0.001,
    tolerance=0',     -- Optimizer params
    'relu',           -- Activation function
    NULL,             -- Default weight (1)
    FALSE,            -- No warm start
    FALSE,            -- Not verbose
    'zipcode'         -- Grouping column
);

Done.
1 rows affected.
1 rows affected.


mlp_regression
""


# 20. View regression model with grouping

In [141]:
%%sql
SELECT * FROM mlp_regress_group;

2 rows affected.


[(20001, [0.418475835332, -0.0824808485536, 0.215181255493, 0.276090822034, 0.278446514223, -0.23619495331, -0.171707885851, 0.250897033082, -0.162681797272, 0.0592726110788, 0.0253747522631, 0.143667118898, -0.148621888187, 0.0653538461103, 0.536182542335, 0.552718908479, 0.105983039484, 0.228280492229, -0.400277030495, 0.203925805832, -0.377741382405, -0.0804013398103, -0.331949921704, 0.181872166267, -0.289476957601, -0.197745997911, -0.409501606674, -0.185108507551, 0.344106454388, -0.273567717732, -0.0767497209843, 0.264827115066, 0.0945645060746, -0.154756118434, 0.102888959552, 0.0189668958034, -0.00954275300822, 0.34995797155, -0.142898167762, 0.19122838501, 0.0254177229286, 0.215318588618, -0.0824471328056, 0.268311191703, -0.120178931613, -0.0732011741189, 0.240315118476, 0.319381943065, -0.358626200769, 0.373528417486, 0.0203008704592, -0.300636668499, -0.288923650295, 0.0622576777885, 0.322389733764, -0.140635773331, -0.362986948221, -0.352204479613, -0.0700832718257, -0.401940055854, -0.188471109268, 0.367538138156, 0.2865336525, 0.304495694955, -0.212668167011, 0.0310505348836, -0.127903722997, 0.210317676262, 0.011348733686, 0.158049867996, 0.0551303657849, -0.329347942709, -0.0791490059317, 0.344681374236, 0.355725957464, 0.111343694661, -0.16845442313, 0.193167928264, 0.106676655838, -0.111340837437, 0.146708920819, -0.25821619534, -0.0529688863735, 0.27179876023, 0.243333918156, -0.135135659693, -0.214298975388, 0.309839922992, -0.112433334404, 0.133486318758, 0.569989803455, 0.0692228362911, 0.161109594706, 0.170376460906, 0.0625288616861, 0.331095975308, 0.0304139707824, 0.329356172126, 0.160733813748, 0.352267621192, -0.123402870539, -0.331653414828, 0.461350242065, 0.442848285723, -0.33331411622, 0.229600104429, 0.110175816309, -0.0365460378314, 0.0721032420867, -0.149685263071, 0.223351158583, -0.12906958588, -0.0558187990368, -0.248074303961, -0.25039930268, -0.196042548065, 0.0227845999626, -0.043912968357, -0.279543037508, 0.253740272865, -0.0750773657413, -0.292013614361, 0.0701996143948, 0.0777107788713, 0.504268483677, 0.339713047868, 0.269746295505, -0.162039022616, 0.141883048842, -0.153441164628, -0.286454262191, -0.334793676562, 0.19168601595, -0.276669547362, -0.316860743488, 0.388750415109, -0.292482519008, 0.28126811857, -0.372282881212, 0.301352479505, -0.332267709825, 0.461471159116, -0.418590744135, -0.0405704502936, -0.274712715975, -0.312962155492, 0.122597272754, 0.2916555356, 0.032005703799, 0.350921818803, 0.233946654106, 0.202837945086, -0.414230346, -0.171691834124, -0.0976454453456, -0.329376394736, -0.449306666217, -0.335399336824, -0.148855755838, 0.285590367147, 0.170685969074, 0.313022250573, -0.463338310074, -0.377242350716, 0.511339340448, -0.173648564996, 0.304532888421, -0.392970224473, 0.497326291479, -0.456829293274, 0.289370283382, -0.442560521876, -0.379995508985, 0.279081255757, 0.143022600044, -0.141036886336, -0.154500931503, -0.193157646093, 0.34358891298, 0.337737839662, -0.0953343446041, 0.0409877781292, 0.0661783621843, -0.0604893324563, 0.169792589295, -0.312658041356, 0.0475130638026, 0.171535770922, -0.178346899797, 0.371779196069, 0.0366562030307, 0.271440801493, 0.216978581738, 0.118766103044, -0.396338113354, 0.520512829834, 0.260240099519, 0.400032341842, 0.117958966008, 0.432065688603, 0.374410429241, 0.363946948142, 0.43063164801, 0.404223535377, 0.154150781717, -0.175685382989, -0.435966071095, 0.431264670542, 0.473001473708, -0.436357823118, 0.165033955146, -0.196763025683, 0.092109073823, -0.236855991365, 0.18200021215, -0.14301621348, -0.108387683449, -0.215672578658, -0.367678133537, 0.126635545935, -0.241135998022, 0.0926978679184, 0.0336727074675, 0.347183377668, -0.234718837618, -0.0142494722154, 0.171236123245, -0.315613629509, 0.00455895441364, 0.0454395909606, 0.34698683075, -0.0651593124215, -0.141272274176, 0.243078974127, -0.03092644296, -0.361118852451, -0.094261864514, 0.0500794669768, 0.243792435547, -0.203123877679, 0.155726456925, -0.

# 21. Regression with n_tries and grouping
Now, use the n_tries optimizer parameter to learn and choose the best model among n_tries number of models learnt by the algorithm for each zipcode.

In [142]:
%%sql
DROP TABLE IF EXISTS mlp_regress_group, mlp_regress_group_summary, mlp_regress_group_standardization;

-- Set seed so results are reproducible
SELECT setseed(0);
SELECT madlib.mlp_regression(
    'lin_housing',    -- Source table
    'mlp_regress_group',    -- Desination table
    'x',              -- Input features
    'y',              -- Dependent variable
    ARRAY[25,25],     -- Number of units per layer
    'learning_rate_init=0.001,
    n_iterations=50,
    n_tries=3,
    lambda=0.001,
    tolerance=0',     -- Optimizer params, with n_tries
    'relu',           -- Activation function
    NULL,             -- Default weight (1)
    FALSE,            -- No warm start
    FALSE,            -- Not verbose
    'zipcode'         -- Grouping column
);

Done.
1 rows affected.
1 rows affected.


mlp_regression
""


# 22. View regression model with n_tries and grouping

In [143]:
%%sql
SELECT * FROM mlp_regress_group ORDER BY zipcode;

2 rows affected.


[(20001, [-0.272696022176, -0.233047946739, 0.050641423477, 0.0480563947444, -0.330648174261, 0.0446962557675, -0.420254650761, 0.221951792183, -0.272150735063, -0.231312029444, 0.103804832915, -0.00401016902576, -0.381233082587, -0.381272413497, -0.257174622704, 0.13485006549, 0.212947464423, 0.371040768992, -0.270346195534, 0.117851442447, 0.266919018035, 0.356621597417, 0.239175949132, -0.28435277863, 0.265232877094, -0.456512090924, -0.126442090813, 0.0655157695241, 0.396983204086, -0.202836962122, 0.677575450335, -0.366063937858, 0.194178940124, -0.111461256869, 0.211077406348, 0.176702824116, -0.109073703169, 0.0110453360433, 0.00356852079065, -0.288880126462, 0.0114403097399, -0.480533454616, 0.342856127105, 0.326137015831, -0.441160625964, 0.164584696005, -0.245465618701, 0.281105049506, 0.191732987889, -0.0475931414553, -0.0718135249828, -0.0196780834452, -0.0798037194663, -0.192210340825, 0.190106603929, -0.190272185637, -0.145563129496, -0.30856399651, 0.165125176616, -0.249063806662, 0.0380034485867, 0.0629338203929, -0.2626126246, -0.265746719994, -0.269496112291, 0.172034019713, 0.295439066034, -0.130761561844, -0.162280308351, -0.0589635218684, 0.0560224346812, -0.376127891034, 0.0102146395822, -0.142259381093, 0.243850417764, 0.142893693397, -0.315590422784, -0.408577214871, 0.0789227502805, 0.19851796495, -0.0027991336311, 0.346078503608, -0.181199612615, 0.201037589796, -0.248542179788, 0.408834582832, 0.415256196195, -0.0861657632293, -0.317525407748, 0.123092571644, 0.206236763629, 0.161936342638, -0.18573819552, 0.102694311796, 0.367810657138, -0.101256504469, 0.0493758274674, 0.198640824519, 0.144827527845, 0.146294799589, -0.356173607724, -0.28997328165, 0.279400756478, 0.20535538779, -0.0648626933847, 0.178407900294, -0.136076451152, 0.109782877732, -0.0133597556628, 0.374491247061, -0.0769322365212, -0.275949882494, 0.318566658863, 0.227803769594, 0.25174007072, -0.43224144193, 0.0956911590803, 0.342829505462, 0.0217346220896, 0.252460257329, -0.0813103920537, -0.375515737922, 0.0715297456271, 0.119098559735, 0.234101315598, -0.0590282313148, -0.311183178736, -0.179631984402, -0.235593266757, 0.17699951173, 0.32100253442, -0.171736785016, 0.399504962653, 0.214347025802, 0.32780453609, -0.0263193827062, -0.184428899868, -0.158094331491, -0.303996882873, 0.251933803175, -0.226760004707, -0.00580481952964, 0.305715243343, -0.322440644276, -0.21135828379, 0.206822991829, -0.295478484693, 0.327606796885, 0.0970986212743, 0.0668074229984, 0.195917788914, -0.32289661279, 0.0815375796396, -0.134894280456, 0.233884854903, -0.118493870251, 0.226356927403, 0.236706591909, 0.00234166915147, 0.329672312723, 0.0455182596209, -0.00353346570482, -0.173319919579, 0.0315356571276, -0.265169148082, -0.170982760236, -0.366961027524, -0.057638674085, 0.0985632161802, -0.331356634884, -0.148995677579, 0.175881462632, 0.11372148149, -0.240235839446, 0.315721692608, 0.325586642404, 0.403723411691, 0.326157828953, 0.235155442456, 0.0820222686884, -0.0741775165281, 0.0562635787751, 0.0718504575077, -0.231533679347, 0.270517338218, -0.0524462121432, -0.0720109089765, 0.134505845506, -0.183816925979, -0.317800891764, 0.157535571314, 0.333333826435, -0.0113162213203, 0.267112990525, -0.0297634735296, 0.620391598966, -0.28133407317, 0.100872133426, -0.414333939676, 0.383027710599, 0.049605112612, 0.264189266763, -0.16739996827, -0.304517817248, 0.032839361994, -0.334641700916, -0.563291290974, 0.393245491484, -0.171499152421, 0.123657145846, -0.0873824680127, 0.285763082916, -0.0443397263519, 0.344143607831, 0.391149888403, -0.128200447735, -0.0965898856509, -0.13767635803, -0.466055444379, 0.0747834996134, 0.0154592426142, 0.118157505449, 0.00435272021135, -0.468673153221, 0.0555557966277, 0.324313053352, 0.275839698657, 0.142423365352, -0.138042275273, -0.307568221253, -0.0829118814138, 0.314119214356, 0.109351600655, 0.305129729582, -0.326727718747, -0.0637759424951, -0.211961722822, -0.266542348861, -0.256138572099, 0.050790649

# 23. Regression with warm start and grouping
Next use warm_start to improve the models that are already present in the mlp_regress_group table.

In [144]:
%%sql
SELECT madlib.mlp_regression(
    'lin_housing',    -- Source table
    'mlp_regress_group',    -- Desination table
    'x',              -- Input features
    'y',              -- Dependent variable
    ARRAY[25,25],     -- Number of units per layer
    'learning_rate_init=0.001,
    n_iterations=450,
    lambda=0.001,
    tolerance=0',     -- Optimizer params
    'relu',           -- Activation function
    NULL,             -- Default weight (1)
    TRUE,             -- Warm start
    FALSE,            -- Not verbose
    'zipcode'         -- Grouping column
);

1 rows affected.


mlp_regression
""


# 24. View regression model with warm start and grouping

In [145]:
%%sql
SELECT * FROM mlp_regress_group ORDER BY zipcode;

2 rows affected.


[(20001, [-0.27282262343, -0.23210601079, 0.0503444792287, 0.047950538862, -0.329400135581, 0.044515601912, -0.418644439558, 0.221468540727, -0.270586341925, -0.230060367515, 0.103473652327, -0.00390558165496, -0.379780589234, -0.37932331672, -0.256269147481, 0.139183589605, 0.212086770844, 0.354023752137, -0.271562474291, 0.119684076293, 0.265840181589, 0.357489167238, 0.235468083997, -0.298253325173, 0.301138146458, -0.456975922161, -0.128240002881, 0.0675599343229, 0.423389951539, -0.178296738402, 0.684688304376, -0.364584376303, 0.171431257928, -0.114801015726, 0.214014535468, 0.175988624551, -0.104842583515, 0.000182380331971, -0.0251023356745, -0.233489313103, 0.00760380588532, -0.482381493088, 0.345260631439, 0.355302617268, -0.404016963607, 0.160258320639, -0.244473492866, 0.281633235779, 0.193212182976, -0.0496549226825, -0.0715232682296, -0.0218526920829, -0.0799754595071, -0.194045444548, 0.188833205593, -0.187248997512, -0.14272064697, -0.309570982642, 0.161720278448, -0.24450926141, 0.0447913520979, 0.0626794537333, -0.259203428335, -0.26828864228, -0.26479083919, 0.171338690577, 0.297860977447, -0.120344855526, -0.157860530854, -0.0552850845592, 0.0521799820148, -0.378223673393, 0.0137893744594, -0.140256947523, 0.247009181169, 0.144560931505, -0.314314865686, -0.40616483272, 0.0777161006505, 0.198603251414, -0.00278782005973, 0.345567378147, -0.182392518619, 0.200467461556, -0.249393336797, 0.406294491656, 0.412690150068, -0.0849298387038, -0.316817639366, 0.122699596246, 0.211676084318, 0.161281826412, -0.197773017865, 0.0993812916737, 0.369221984704, -0.100847244734, 0.0520742087039, 0.192980696982, 0.131797527255, 0.175657241493, -0.357631970072, -0.29169921326, 0.281169420227, 0.229893646759, -0.0428737294798, 0.179120214833, -0.135526455716, 0.0997532118612, -0.0136855103856, 0.373357376253, -0.0766212908977, -0.274454792649, 0.317328236171, 0.217172846926, 0.27399876383, -0.430874155695, 0.0949246410807, 0.341823605448, 0.0369312664329, 0.264220558045, -0.081336042182, -0.373997973976, 0.0712571037461, 0.118569222562, 0.233203085338, -0.0587896503113, -0.309877471668, -0.179142979248, -0.235380722804, 0.175883505435, 0.319657139374, -0.171090620452, 0.397938201973, 0.21340235615, 0.326735182919, -0.0270564995142, -0.183683472962, -0.161495033696, -0.302375689766, 0.250523039242, -0.225843483441, -0.00617385245549, 0.304550898265, -0.326259108417, -0.200296781729, 0.206379547134, -0.293891721343, 0.325890176939, 0.10111815821, 0.0712727879178, 0.203959017745, -0.321591525444, 0.0800339844078, -0.136852858471, 0.235443331654, -0.118014940325, 0.22794583063, 0.230803209898, 0.00362427700506, 0.328588366391, 0.0428304880618, -0.0060229794205, -0.170115597754, 0.0385155856994, -0.260209266853, -0.17087973072, -0.365477840104, -0.0574695885202, 0.0983575015883, -0.330210013093, -0.148393465087, 0.174977923473, 0.114232967112, -0.236231731507, 0.316084830231, 0.324463341837, 0.402284297013, 0.324646900997, 0.234525192887, 0.0806402332749, -0.0767493587264, 0.0560361719772, 0.074482906619, -0.229227779472, 0.268053873727, -0.0522342344226, -0.073089938745, 0.127014186125, -0.18511072107, -0.322293507282, 0.158268926491, 0.333356638143, -0.0126405675957, 0.263927175673, -0.027506863769, 0.632894055319, -0.280196973779, 0.0771129896291, -0.419320963864, 0.388141268671, 0.0494046180812, 0.269783147465, -0.183838610047, -0.335014511141, 0.0902650422376, -0.339950826138, -0.567676257575, 0.398317751209, -0.136181372518, 0.165437071953, -0.0955521794772, 0.284608082308, -0.0532684398557, 0.3433874324, 0.38893414588, -0.127682285649, -0.0968342757879, -0.145256845237, -0.451598621623, 0.0718635277772, 0.0160315475443, 0.118314723333, 0.00370033905614, -0.475481108212, 0.0671415575378, 0.32288332363, 0.274724806503, 0.137053148998, -0.13672283172, -0.307086591022, -0.0825767671901, 0.312088100951, 0.109451124968, 0.297837457253, -0.313354828152, -0.0627566694884, -0.21034351077, -0.266226537494, -0.25061318634, 0.05561968

# 25. Prediction for classification

In the following examples we will use the training data set for prediction as well, which is not usual but serves to show the syntax. First we will test the classification example. The prediction is in the the estimated_class_text column with the actual value in the class_text column.

In [146]:
%%sql
DROP TABLE IF EXISTS mlp_prediction;

SELECT madlib.mlp_predict(
         'mlp_model',         -- Model table
         'iris_data',         -- Test data table
         'id',                -- Id column in test table
         'mlp_prediction',    -- Output table for predictions
         'response'           -- Output classes, not probabilities
     );

SELECT * FROM mlp_prediction JOIN iris_data USING (id) ORDER BY id;

Done.
1 rows affected.
52 rows affected.


id,estimated_class_text,attributes,class_text,class,state
1,Iris_setosa,"[Decimal('5.0'), Decimal('3.2'), Decimal('1.2'), Decimal('0.2')]",Iris_setosa,1,Alaska
2,Iris_setosa,"[Decimal('5.5'), Decimal('3.5'), Decimal('1.3'), Decimal('0.2')]",Iris_setosa,1,Alaska
3,Iris_setosa,"[Decimal('4.9'), Decimal('3.1'), Decimal('1.5'), Decimal('0.1')]",Iris_setosa,1,Alaska
4,Iris_setosa,"[Decimal('4.4'), Decimal('3.0'), Decimal('1.3'), Decimal('0.2')]",Iris_setosa,1,Alaska
5,Iris_setosa,"[Decimal('5.1'), Decimal('3.4'), Decimal('1.5'), Decimal('0.2')]",Iris_setosa,1,Alaska
6,Iris_setosa,"[Decimal('5.0'), Decimal('3.5'), Decimal('1.3'), Decimal('0.3')]",Iris_setosa,1,Alaska
7,Iris_setosa,"[Decimal('4.5'), Decimal('2.3'), Decimal('1.3'), Decimal('0.3')]",Iris_setosa,1,Alaska
8,Iris_setosa,"[Decimal('4.4'), Decimal('3.2'), Decimal('1.3'), Decimal('0.2')]",Iris_setosa,1,Alaska
9,Iris_setosa,"[Decimal('5.0'), Decimal('3.5'), Decimal('1.6'), Decimal('0.6')]",Iris_setosa,1,Alaska
10,Iris_setosa,"[Decimal('5.1'), Decimal('3.8'), Decimal('1.9'), Decimal('0.4')]",Iris_setosa,1,Alaska


Count the misclassifications

In [147]:
%%sql
SELECT COUNT(*) FROM mlp_prediction JOIN iris_data USING (id) 
WHERE mlp_prediction.estimated_class_text != iris_data.class_text;

1 rows affected.


count
0


# 26 Prediction for classification using grouping
Prediction based on grouping using the state column:

In [148]:
%%sql
DROP TABLE IF EXISTS mlp_prediction;

SELECT madlib.mlp_predict(
         'mlp_model_group',   -- Model table
         'iris_data',         -- Test data table
         'id',                -- Id column in test table
         'mlp_prediction',    -- Output table for predictions
         'response'           -- Output classes, not probabilities
     );

SELECT * FROM mlp_prediction JOIN iris_data USING (state,id) ORDER BY state, id;

Done.
1 rows affected.
52 rows affected.


state,id,estimated_class_text,attributes,class_text,class
Alaska,1,Iris_setosa,"[Decimal('5.0'), Decimal('3.2'), Decimal('1.2'), Decimal('0.2')]",Iris_setosa,1
Alaska,2,Iris_setosa,"[Decimal('5.5'), Decimal('3.5'), Decimal('1.3'), Decimal('0.2')]",Iris_setosa,1
Alaska,3,Iris_setosa,"[Decimal('4.9'), Decimal('3.1'), Decimal('1.5'), Decimal('0.1')]",Iris_setosa,1
Alaska,4,Iris_setosa,"[Decimal('4.4'), Decimal('3.0'), Decimal('1.3'), Decimal('0.2')]",Iris_setosa,1
Alaska,5,Iris_setosa,"[Decimal('5.1'), Decimal('3.4'), Decimal('1.5'), Decimal('0.2')]",Iris_setosa,1
Alaska,6,Iris_setosa,"[Decimal('5.0'), Decimal('3.5'), Decimal('1.3'), Decimal('0.3')]",Iris_setosa,1
Alaska,7,Iris_setosa,"[Decimal('4.5'), Decimal('2.3'), Decimal('1.3'), Decimal('0.3')]",Iris_setosa,1
Alaska,8,Iris_setosa,"[Decimal('4.4'), Decimal('3.2'), Decimal('1.3'), Decimal('0.2')]",Iris_setosa,1
Alaska,9,Iris_setosa,"[Decimal('5.0'), Decimal('3.5'), Decimal('1.6'), Decimal('0.6')]",Iris_setosa,1
Alaska,10,Iris_setosa,"[Decimal('5.1'), Decimal('3.8'), Decimal('1.9'), Decimal('0.4')]",Iris_setosa,1


# 27.  Prediction for regression

In [149]:
%%sql
DROP TABLE IF EXISTS mlp_regress_prediction;

SELECT madlib.mlp_predict(
         'mlp_regress',               -- Model table
         'lin_housing',               -- Test data table
         'id',                        -- Id column in test table
         'mlp_regress_prediction',    -- Output table for predictions
         'response'                   -- Output values, not probabilities
     );

SELECT *, ABS(y-estimated_y) as abs_diff FROM lin_housing JOIN mlp_regress_prediction USING (id) ORDER BY id;

Done.
1 rows affected.
20 rows affected.


id,x,zipcode,y,estimated_y,abs_diff
1,"[Decimal('1'), Decimal('0.00632'), Decimal('18.00'), Decimal('2.310'), Decimal('0'), Decimal('0.5380'), Decimal('6.5750'), Decimal('65.20'), Decimal('4.0900'), Decimal('1'), Decimal('296.0'), Decimal('15.30'), Decimal('396.90'), Decimal('4.98')]",94016,24.0,24.0336254813,0.0336254813462
2,"[Decimal('1'), Decimal('0.02731'), Decimal('0.00'), Decimal('7.070'), Decimal('0'), Decimal('0.4690'), Decimal('6.4210'), Decimal('78.90'), Decimal('4.9671'), Decimal('2'), Decimal('242.0'), Decimal('17.80'), Decimal('396.90'), Decimal('9.14')]",94016,21.6,21.7774199267,0.177419926678
3,"[Decimal('1'), Decimal('0.02729'), Decimal('0.00'), Decimal('7.070'), Decimal('0'), Decimal('0.4690'), Decimal('7.1850'), Decimal('61.10'), Decimal('4.9671'), Decimal('2'), Decimal('242.0'), Decimal('17.80'), Decimal('392.83'), Decimal('4.03')]",94016,34.7,34.6146077773,0.0853922227406
4,"[Decimal('1'), Decimal('0.03237'), Decimal('0.00'), Decimal('2.180'), Decimal('0'), Decimal('0.4580'), Decimal('6.9980'), Decimal('45.80'), Decimal('6.0622'), Decimal('3'), Decimal('222.0'), Decimal('18.70'), Decimal('394.63'), Decimal('2.94')]",94016,33.4,34.1984519925,0.798451992539
5,"[Decimal('1'), Decimal('0.06905'), Decimal('0.00'), Decimal('2.180'), Decimal('0'), Decimal('0.4580'), Decimal('7.1470'), Decimal('54.20'), Decimal('6.0622'), Decimal('3'), Decimal('222.0'), Decimal('18.70'), Decimal('396.90'), Decimal('5.33')]",94016,36.2,35.0776142772,1.12238572283
6,"[Decimal('1'), Decimal('0.02985'), Decimal('0.00'), Decimal('2.180'), Decimal('0'), Decimal('0.4580'), Decimal('6.4300'), Decimal('58.70'), Decimal('6.0622'), Decimal('3'), Decimal('222.0'), Decimal('18.70'), Decimal('394.12'), Decimal('5.21')]",94016,28.7,29.3757815065,0.675781506516
7,"[Decimal('1'), Decimal('0.08829'), Decimal('12.50'), Decimal('7.870'), Decimal('0'), Decimal('0.5240'), Decimal('6.0120'), Decimal('66.60'), Decimal('5.5605'), Decimal('5'), Decimal('311.0'), Decimal('15.20'), Decimal('395.60'), Decimal('12.43')]",94016,22.9,22.6565999866,0.243400013359
8,"[Decimal('1'), Decimal('0.14455'), Decimal('12.50'), Decimal('7.870'), Decimal('0'), Decimal('0.5240'), Decimal('6.1720'), Decimal('96.10'), Decimal('5.9505'), Decimal('5'), Decimal('311.0'), Decimal('15.20'), Decimal('396.90'), Decimal('19.15')]",94016,27.1,24.0556266918,3.04437330819
9,"[Decimal('1'), Decimal('0.21124'), Decimal('12.50'), Decimal('7.870'), Decimal('0'), Decimal('0.5240'), Decimal('5.6310'), Decimal('100.00'), Decimal('6.0821'), Decimal('5'), Decimal('311.0'), Decimal('15.20'), Decimal('386.63'), Decimal('29.93')]",94016,16.5,16.4829549347,0.0170450652555
10,"[Decimal('1'), Decimal('0.17004'), Decimal('12.50'), Decimal('7.870'), Decimal('0'), Decimal('0.5240'), Decimal('6.0040'), Decimal('85.90'), Decimal('6.5921'), Decimal('5'), Decimal('311.0'), Decimal('15.20'), Decimal('386.71'), Decimal('17.10')]",94016,18.9,18.4922988342,0.407701165755


In [150]:
%%sql
SELECT SQRT(SUM(ABS(y-estimated_y))/COUNT(y)) as rms_error FROM lin_housing 
JOIN mlp_regress_prediction USING (id);

1 rows affected.


rms_error
0.70910039986


# 28.  Prediction for regression with grouping

In [151]:
%%sql
DROP TABLE IF EXISTS mlp_regress_prediction;

SELECT madlib.mlp_predict(
         'mlp_regress_group',         -- Model table
         'lin_housing',             -- Test data table
         'id',                        -- Id column in test table
         'mlp_regress_prediction',    -- Output table for predictions
         'response'                   -- Output values, not probabilities
     );

SELECT * FROM lin_housing JOIN mlp_regress_prediction USING (zipcode, id) ORDER BY zipcode, id;

Done.
1 rows affected.
20 rows affected.


zipcode,id,x,y,estimated_y
20001,12,"[Decimal('1'), Decimal('0.11747'), Decimal('12.50'), Decimal('7.870'), Decimal('0'), Decimal('0.5240'), Decimal('6.0090'), Decimal('82.90'), Decimal('6.2267'), Decimal('5'), Decimal('311.0'), Decimal('15.20'), Decimal('396.90'), Decimal('13.27')]",18.9,18.899621713
20001,13,"[Decimal('1'), Decimal('0.09378'), Decimal('12.50'), Decimal('7.870'), Decimal('0'), Decimal('0.5240'), Decimal('5.8890'), Decimal('39.00'), Decimal('5.4509'), Decimal('5'), Decimal('311.0'), Decimal('15.20'), Decimal('390.50'), Decimal('15.71')]",21.7,21.6998163063
20001,14,"[Decimal('1'), Decimal('0.62976'), Decimal('0.00'), Decimal('8.140'), Decimal('0'), Decimal('0.5380'), Decimal('5.9490'), Decimal('61.80'), Decimal('4.7075'), Decimal('4'), Decimal('307.0'), Decimal('21.00'), Decimal('396.90'), Decimal('8.26')]",20.4,20.391032521
20001,15,"[Decimal('1'), Decimal('0.63796'), Decimal('0.00'), Decimal('8.140'), Decimal('0'), Decimal('0.5380'), Decimal('6.0960'), Decimal('84.50'), Decimal('4.4619'), Decimal('4'), Decimal('307.0'), Decimal('21.00'), Decimal('380.02'), Decimal('10.26')]",18.2,18.2037333689
20001,16,"[Decimal('1'), Decimal('0.62739'), Decimal('0.00'), Decimal('8.140'), Decimal('0'), Decimal('0.5380'), Decimal('5.8340'), Decimal('56.50'), Decimal('4.4986'), Decimal('4'), Decimal('307.0'), Decimal('21.00'), Decimal('395.62'), Decimal('8.47')]",19.9,19.9043190662
20001,17,"[Decimal('1'), Decimal('1.05393'), Decimal('0.00'), Decimal('8.140'), Decimal('0'), Decimal('0.5380'), Decimal('5.9350'), Decimal('29.30'), Decimal('4.4986'), Decimal('4'), Decimal('307.0'), Decimal('21.00'), Decimal('386.85'), Decimal('6.58')]",23.1,23.0993595905
20001,18,"[Decimal('1'), Decimal('0.78420'), Decimal('0.00'), Decimal('8.140'), Decimal('0'), Decimal('0.5380'), Decimal('5.9900'), Decimal('81.70'), Decimal('4.2579'), Decimal('4'), Decimal('307.0'), Decimal('21.00'), Decimal('386.75'), Decimal('14.67')]",17.5,17.4974706738
20001,19,"[Decimal('1'), Decimal('0.80271'), Decimal('0.00'), Decimal('8.140'), Decimal('0'), Decimal('0.5380'), Decimal('5.4560'), Decimal('36.60'), Decimal('3.7965'), Decimal('4'), Decimal('307.0'), Decimal('21.00'), Decimal('288.99'), Decimal('11.69')]",20.2,20.1990236374
20001,20,"[Decimal('1'), Decimal('0.72580'), Decimal('0.00'), Decimal('8.140'), Decimal('0'), Decimal('0.5380'), Decimal('5.7270'), Decimal('69.50'), Decimal('3.7965'), Decimal('4'), Decimal('307.0'), Decimal('21.00'), Decimal('390.95'), Decimal('11.28')]",18.2,18.2004361872
94016,1,"[Decimal('1'), Decimal('0.00632'), Decimal('18.00'), Decimal('2.310'), Decimal('0'), Decimal('0.5380'), Decimal('6.5750'), Decimal('65.20'), Decimal('4.0900'), Decimal('1'), Decimal('296.0'), Decimal('15.30'), Decimal('396.90'), Decimal('4.98')]",24.0,23.9999471447
